In [1]:
pip install app-store-scraper pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 644.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does no

In [10]:
!pip install playwright pandas
!playwright install


In [13]:
pip install nest_asyncio


Note: you may need to restart the kernel to use updated packages.


In [14]:
import nest_asyncio
nest_asyncio.apply()

# Call the async function directly
await main()


Scraping completed. Data saved to 'elsa_app_reviews.csv'.
Empty DataFrame
Columns: []
Index: []


In [20]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd
import nest_asyncio

# Apply asyncio patch for Jupyter environments
nest_asyncio.apply()

async def scrape_ios_web_reviews(app_url, max_reviews=100):
    """
    Scrapes reviews from the web-based App Store for an iOS app.

    Args:
        app_url (str): The URL of the app's review page on the web-based App Store.
        max_reviews (int): Maximum number of reviews to scrape.

    Returns:
        List[Dict]: List of dictionaries containing scraped reviews.
    """
    reviews_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        print(f"Navigating to {app_url}...")
        await page.goto(app_url, timeout=60000)

        # Scroll to load reviews dynamically
        print("Scrolling to load reviews...")
        for _ in range(15):  # Adjust the number of scrolls if necessary
            await page.mouse.wheel(0, 3000)
            await asyncio.sleep(2)

        # Find all review blocks
        review_blocks = await page.query_selector_all('div.we-customer-review')  # Update this if the structure changes
        print(f"Found {len(review_blocks)} reviews on the page.")

        # Parse each review block
        for idx, review in enumerate(review_blocks[:max_reviews]):
            try:
                user_name = await review.query_selector("span.we-truncate__text")
                user_name = await user_name.inner_text() if user_name else "Unknown"

                rating = await review.query_selector("figure > div > span")
                rating = len(await rating.query_selector_all("span[aria-hidden=true]")) if rating else "Unknown"

                review_text = await review.query_selector("blockquote")
                review_text = await review_text.inner_text() if review_text else "No Review"

                date = await review.query_selector("time")
                date = await date.inner_text() if date else "Unknown"

                reviews_data.append({
                    "User": user_name,
                    "Rating": rating,
                    "Review": review_text,
                    "Date": date
                })

                # Debug log for each review
                print(f"Review {idx + 1}: {user_name}, Rating: {rating}, Date: {date}")
            except Exception as e:
                print(f"Error parsing review {idx + 1}: {e}")

        await browser.close()

    return reviews_data

async def main():
    APP_URL = "https://apps.apple.com/us/app/elsa-speak-english-learning/id1080859478"  # Web-based URL for reviews
    reviews = await scrape_ios_web_reviews(APP_URL, max_reviews=100)

    if reviews:
        # Save to CSV
        df = pd.DataFrame(reviews)
        df.to_csv("/Users/qbs/elsa_app_reviews.csv", index=False)
        print("Scraping completed. Data saved to 'elsa_app_reviews.csv'.")
        print(df.head())
    else:
        print("No reviews were scraped. Please check the page structure or selectors.")

# Run the script
await main()


Navigating to https://apps.apple.com/us/app/elsa-speak-english-learning/id1080859478...
Scrolling to load reviews...
Found 0 reviews on the page.
No reviews were scraped. Please check the page structure or selectors.
